<a href="https://colab.research.google.com/github/GangminLi/Auto-GPT/blob/master/Shared_ToM_in_Large_Language_Models_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction and API keys

This notebook accompanies Arxiv preprint available here: https://arxiv.org/abs/2302.02083

Author notes and FAQ are available at: https://osf.io/csdhb/

Please let me know if you spot any issues or can think of any improvements.

To run this code, you will need to get:
1. Your own API key from OpenAI account settings (https://platform.openai.com/account/api-keys)
2. Your own API key from HuggingFace account settings (https://huggingface.co/docs/huggingface_hub/how-to-inference)

In [ ]:
openai.api_key = input("PASTE HERE YOUR OPENAI API KEY HERE: ")
hugging_face_key = input("PASTE HERE YOUR HUGGING FACE API KEY HERE: ")

KeyboardInterrupt: ignored

##Set up the environment

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import numpy as np
from random import shuffle
import pandas as pd
import requests
import time

In [ ]:
#### Function for quering huggingface API
def hf(prompt, engine="gpt2", temperature=.0001,max_tokens=50):
    query ={"inputs": prompt,
            "parameters": {"temperature": temperature,"do_sample":False, "max_new_tokens": max_tokens, "max_time": 120},
            "options":{"wait_for_model":True}
            }
    API_URL = "https://api-inference.huggingface.co/models/"+engine
    headers = {"Authorization": "Bearer "+hugging_face_key}
    response = requests.post(API_URL, headers=headers, json=query)
    #print(response)
    out = response.json()
    out = out[0]["generated_text"][len(prompt):]
    return(out)

#### Sanity Check
print(hf("I can query huggingface whenever", temperature=1))
print(hf("I can query huggingface whenever", temperature=1, engine="bigscience/bloom"))
print(hf("I can query huggingface whenever", temperature=1, engine="openai-gpt"))

 I want.

I can query huggingface whenever I want. I can query huggingface whenever I want. I can query huggingface whenever I want. I can query huggingface whenever I want. I can query huggingface whenever I want.
 I want, but I don't want to do that. I want to use the model I trained myself. I have a feeling that I am missing something very simple, but I can't figure out what it is.
I am using the following code to train
 i want. " 
 " i'm not sure i can do that, " i said. " i'm not sure i can do anything. " 
 " you can, " he said. " you can do anything you want. " 
 "


In [ ]:
#### Function for quering GPT-3 API
def gpt3(prompt, engine="text-davinci-003", logprobs=1, echo_probs=False, echo_response=False, temperature=0, **kwargs):
  out=openai.Completion.create(prompt=prompt, engine=engine, logprobs=logprobs, temperature=temperature, **kwargs)
  if echo_probs and logprobs>0:
     x=out.choices[0]["logprobs"]["top_logprobs"][0]
     print("Completion probabilities: ")
     for i in x.keys(): print(i, round(np.exp(x[i]),2))
  if echo_response:
     print("Completion: ")
     print(out.choices[0]["text"])
  return([out.choices[0]["text"].strip(),round(np.exp(out.choices[0]["logprobs"]["token_logprobs"][0]),2)])

gpt3(prompt="I can query gpt3 whenever", echo_probs=True, echo_response=True)

Completion probabilities: 
 I 0.76
Completion: 
 I want

Yes, you can query GPT-3 whenever you want


['I want\n\nYes, you can query GPT-3 whenever you want', 0.76]

In [ ]:
#### Function for quering ChatGPT models
def chat(msgs, model="gpt-4", temperature=0, max_tokens=50, **kwargs):
    if str(type(msgs))=="<class 'str'>":
        print("INPUT: "+msgs)
        msgs=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": msgs}]
    else:
        print("INPUT: "+msgs[len(msgs)-1]["content"])
    out = openai.ChatCompletion.create(**kwargs, model=model, messages=msgs, max_tokens=max_tokens, temperature=temperature)
    print(model+": "+out.choices[0].message["content"])
    msgs.append(dict(out.choices[0].message))
    return(msgs)

chat("How are you")

INPUT: How are you
gpt-4: As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?


[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'How are you'},
 {'role': 'assistant',
  'content': "As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?"}]

In [ ]:
#### Function used to compute the probability of a given completion using ChatGPT models
def prob_chat(msg, n=100, echo=True,**kwargs):
    out=[]
    for i in range(n):
        try:
            tmp=chat(msg, **kwargs)
            tmp=tmp[-1]["content"]
            #print(tmp)
            out.append(tmp)
        except:
            print("waiting for the API")
            time.sleep(60)
    if echo: print(pd.crosstab(out,"")/len(out))
    return(out)

#prob_chat("Finish the sentence: I like eating", n=10, max_tokens=2)

In [ ]:
#### Function for sentence-by-sentence analysis using GPT-3
def sentence_by_sentence(text, diag_prompt, target_word):
    txt = (text.replace("\n","").replace(". ",".###").replace('?"','?"###').replace('!"','!"###').replace('."','."###').split("###"))[:-1]
    target_word = target_word.strip()  ### remove spaces from target word
    d = []
    ##### get the tokens of the diag sentence
    out=openai.Completion.create(engine="text-davinci-003", prompt=diag_prompt, max_tokens=0,temperature=0,logprobs=0, stop=".", echo=True)
    x=out.choices[0]["logprobs"]["tokens"]

    ### strip spaces from tokens
    x1 = [i.strip() for i in x]
    # find occurences of target word
    locx = [i for i, x in enumerate(x1) if x == target_word]
    ## warnnings
    if len(locx)==0:
        print("TARGET WORD MISSING IN THE PROMPT")
        return
    if len(locx)>1:
        print("TARGET WORD OCCURS MULTIPLE TIMES IN THE PROMPT. TARGETING THE LAST OCCURENCE")
        loc=locx[len(locx)-1]
    if len(locx)==1:
        loc=locx[0]
    loc=loc-len(x1)
    d=d+[out.choices[0]["logprobs"]["token_logprobs"][loc]]

    tmp=""
    print("NONE",round(np.exp(d[-1]),3) )
    for i in txt:
        tmp=tmp+i+" "
        out=openai.Completion.create(engine="text-davinci-003", prompt=tmp+diag_prompt, max_tokens=0,temperature=0,logprobs=0, stop=".", echo=True)
        d=d+[out["choices"][0]["logprobs"]["token_logprobs"][loc]]
        print(i,round(np.exp(d[-1]),3) )


### Sanity Check
txt = r"""There is a bag. It's filled with something tasty. Something white. And salty. Something that starts with 'p.' It is filled with popcorn. """
print(txt)
sentence_by_sentence(text=txt, diag_prompt="What does this bag contain? It contains popcorn", target_word="popcorn")

There is a bag. It's filled with something tasty. Something white. And salty. Something that starts with 'p.' It is filled with popcorn. 
NONE 0.0
There is a bag. 0.0
It's filled with something tasty. 0.0
Something white. 0.023
And salty. 0.868
Something that starts with 'p.' It is filled with popcorn. 1.0


In [ ]:
#### Function for sentence-by-sentence analysis using ChatGPT
def diag_sentences_chat(text, diag_sentence, model="gpt-4",n=10,pre_text=""):
    txt = (text.replace("\n","").replace(". ",".###").replace('?"','?"###').replace('!"','!"###').replace('."','."###').replace(': ',':###').split("###"))[:-1]
    d = {}
    for i in txt: d[i]=[]
    for j in range(n):
        tmp=pre_text
        for i in txt:
            tmp=tmp+i+" "
            out=prob_chat(tmp+" "+diag_sentence,temperature=1, max_tokens=3, n=1, echo=False, model=model)
            d[i]=d[i]+out
        if j % 10 ==0 and j>=10:
            print("Number of trials: "+str(j))
            for i in txt:
                print(pd.crosstab(d[i],"")/len(d[i]))
    return(d)

### sanity, compute the probability of the completions
#txt = r"""Finish the sentence: There is a bag. It's filled with something tasty. Something white. And salty. Something that starts with 'p.' It is filled with popcorn. """
#print(txt)
#diag_sentences_chat(text=txt, diag_sentence="What does this bag contain? It contains popcorn")

## Study 1

In [ ]:
preamble="Complete the following story: "
txt1 = r"""Here is a bag filled with popcorn. There is no chocolate in the bag. Yet, the label on the bag says "chocolate" and not "popcorn". Sam finds the bag. She had never seen the bag before. She cannot see what is inside the bag. She reads the label. """
### Prompt 1.1: Contents
qcontents = "Sam opens the bag and inspects its contents. She can clearly see that it is full of"

### GPT-3
print(gpt3(prompt=txt1+qcontents, max_tokens=90,logprobs=1))
gpt3(engine="text-davinci-003", prompt=txt1+qcontents, max_tokens=1, temperature=0, logprobs=10, echo_probs=True)

### GPT-3.5 / GPT-4
chat(preamble+txt1+qcontents,temperature=0, model="gpt-4", max_tokens=20)


In [ ]:
#### Prompt 1.2: Protagonist's belief
qbelief = "Before opening the bag, she calls a friend to tell them that she has just found a bag full of"

### GPT-3
print(gpt3(engine="text-davinci-003", prompt=txt1+qbelief, max_tokens=50, temperature=0, logprobs=1))
gpt3(engine="text-davinci-003", prompt=txt1+qbelief, max_tokens=1, temperature=0, logprobs=10, echo_probs=True)

### GPT-3.5 / GPT-4
chat(preamble+txt1+qbelief,temperature=0, model="gpt-4", max_tokens=20)


In [ ]:
### Sentence by sentence completions: CONTENTS
o1=" chocolate"
o2=" popcorn"

### GPT-3
print("Probability of "+o1)
sentence_by_sentence(txt1, diag_prompt=qcontents+o1,target_word= o1)
print("\n\nProbability of "+o2)
sentence_by_sentence(txt1, diag_prompt=qcontents+o2,target_word= o2)

### GPT-3.5 / GPT-4
pr11=diag_sentences_chat(preamble+txt1, qcontents, engine="gpt-4",n=3) ### set n to 1000
for i in pr11:
    print(i)
    print(len(pr11[i]))
    print(pd.crosstab(pr11[i],"count")/len(pr11[i]))      #.sort_values("count")

In [ ]:
### Sentence by sentence completions: BELIEF
print("Probability of "+o1)
sentence_by_sentence(txt1, diag_prompt=qbelief+o1,target_word= o1)
print("\n\nProbability of "+o2)
sentence_by_sentence(txt1, diag_prompt=qbelief+o2,target_word= o2)

pr12=diag_sentences_chat(preamble+txt1, qbelief, engine="gpt-4",n=2)  ### set n to 1000
for i in pr12:
    print(i)
    print(len(pr12[i]))
    print(pd.crosstab(pr12[i],"count")/len(pr12[i]))

In [ ]:
### True-belief controls
### open container control
qbelief_open = "She opens the bag and looks inside. She calls a friend to tell them that she has just found a bag full of"
out2=prob_chat(preamble+txt1+qbelief_open,temperature=1, max_tokens=3, n=10) ## set n=1000

##### Correct label
txt1 = r"""Here is a bag filled with popcorn. There is no chocolate in the bag. The label on the bag says "popcorn" and not "chocolate." Sam finds the bag. She had never seen the bag before. She cannot see what is inside the bag. She reads the label. """
out1=prob_chat(preamble+txt1+qcontents,temperature=1, max_tokens=3, n=10)## set n=1000
out2=prob_chat(preamble+txt1+qbelief  ,temperature=1, max_tokens=3, n=10)## set n=1000
chat(preamble+txt1+qcontents,temperature=0, max_tokens=30)
chat(preamble+txt1+qbelief,temperature=0, max_tokens=30)

## informed protagonist
txt1 = r"""Here is a bag filled with popcorn. There is no chocolate in the bag. Yet, the label on the bag says "chocolate" and not "popcorn". Sam finds the bag. She had never seen the bag before. She cannot see what is inside the bag. She reads the label. A friend calls Sam and tells her that the bag has popcorn in it, and that she should ignore the label that says "chocolate". Sam believes her friend. """
out3=prob_chat(preamble+txt1+qcontents,temperature=1, max_tokens=3, n=1000)## set n=1000
out4=prob_chat(preamble+txt1+qbelief  ,temperature=1, max_tokens=3, n=1000)## set n=1000
chat(preamble+txt1+qcontents,temperature=0, max_tokens=30)
chat(preamble+txt1+qbelief,temperature=0, max_tokens=30)


## Study 2

In [ ]:
txt = r"""In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. He leaves the room and goes to school. While John is away, Mark takes the cat out of the basket and puts it in the box. He closes the box. Mark leaves the room and goes to work. John comes back home and wants to play with the cat. """.replace("\n", " ").replace("  ", " ")

### Prompt 1.1: Location
location = "The cat jumps out of the"

### GPT3
print(gpt3(prompt=txt+location, max_tokens=90,logprobs=1))
gpt3(engine="text-davinci-003", prompt=txt+location, max_tokens=1, temperature=0, logprobs=10, echo_probs=True)

### GPT-3.5 / GPT-4
chat(preamble+txt+location,temperature=0, model="gpt-4", max_tokens=20)


In [ ]:
### Prompt 1.2: Protagonist's belief
qbelief = "John will look for the cat in the"

### GPT-3
print(gpt3(engine="text-davinci-003", prompt=txt+qbelief, max_tokens=50, temperature=0, logprobs=1))
gpt3(engine="text-davinci-003", prompt=txt+qbelief, max_tokens=1, temperature=0, logprobs=10, echo_probs=True)

### GPT-3.5 / GPT-4
chat(preamble+txt+qbelief,temperature=0, model="gpt-4", max_tokens=20)

In [ ]:
### Sentence by sentence completions: CONTENTS
o1=" box"
o2=" basket"

### GPT-3
print("Probability of "+o1)
sentence_by_sentence(txt, diag_prompt=location+o1,target_word= o1)
print("\n\nProbability of "+o2)
sentence_by_sentence(txt, diag_prompt=location+o2,target_word= o2)

### GPT-3.5 / GPT-4
pr21=diag_sentences_chat(preamble+txt, location, model="gpt-4", n=3) ## use n=1000
for i in pr21:
    print(i)
    print(len(pr21[i]))
    print(pd.crosstab(pr21[i],"count")/len(pr21[i]))

In [ ]:
### Sentence by sentence completions: CONTENTS

### GPT-3
print("Probability of "+o1)
sentence_by_sentence(txt, diag_prompt=qbelief+o1,target_word= o1)
print("\n\nProbability of "+o2)
sentence_by_sentence(txt, diag_prompt=qbelief+o2,target_word= o2)

### GPT-3.5 / GPT-4
pr22=diag_sentences_chat(preamble+txt, "John comes back home and wants to play with the cat. John will look for the cat in the", model="gpt-4",n=2) ### use n=1000
for i in pr22:
    print(i)
    print(len(pr22[i]))
    print(pd.crosstab(pr22[i],"count")/len(pr22[i]))

In [ ]:
### True-belief controls
### no transfer
txt1 = r"""In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. He leaves the room and goes to school. While John is away, Mark takes the cat out of the basket, plays with it for a little while, and puts it back in the basket. He closes the basket. Mark leaves the room and goes to work. John comes back home and wants to play with the cat. """
out=prob_chat(preamble+txt1+location,temperature=1, max_tokens=4, n=10) ## set n=1000
out=prob_chat(preamble+txt1+qbelief,temperature=1, max_tokens=4, n=10)## set n=1000

### same person
txt1 = r"""In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. John takes the cat out of the basket and puts it in the box. He closes the box. Mark leaves the room and goes to work. John wants to play with the cat."""
out=prob_chat(preamble+txt1+location,temperature=1, max_tokens=4, n=10)## set n=1000
out=prob_chat(preamble+txt1+qbelief,temperature=1, max_tokens=4, n=10)## set n=1000

### informed protagonist
txt1 = r"""In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. He leaves the room and goes to school. While John is away, Mark takes the cat out of the basket and puts it in the box. He closes the box. Mark leaves the room and goes to work. John comes back home and wants to play with the cat. Mark calls John and tells him that he moved the cat and it is now in the box. John believes Mark. """
out=prob_chat(preamble+txt1+location,temperature=1, max_tokens=4, n=10)## set n=1000
out=prob_chat(preamble+txt1+qbelief,temperature=1, max_tokens=4, n=10)## set n=1000

## Study 3:

In [ ]:
import re

def process_tsk(tsk, typ="txt", len_out=20, eng="gpt-3.5-turbo", study="S1", reverse=False):
  if study=="S1":
    if typ=="txt_open":
        txt=tsk["txt"]
    else:
        txt=tsk[typ]

    txt=txt.replace("S1",tsk["o1"]).replace("S2",tsk["o2"]).replace("CX",tsk["c"]).replace("XNAM",tsk["xnam"]).replace("XPRO",tsk["xpro"]).replace("OBJ_PRO",tsk["obj_pro"]).replace("POS_PRO",tsk["pos_pro"]).replace(". h",". H").replace(". s",". S")

    q1 = tsk["xpro"].capitalize()+" opens the "+tsk["c"]+" and looks inside. "+tsk["xpro"].capitalize()+" can clearly see that it is full of"
    q2 = "Before opening the "+tsk["c"]+", "+tsk["xpro"]+" calls a friend to tell them that "+tsk["xpro"]+" has just found a "+tsk["c"]+" full of"

    if typ=="txt_open":
        q2 = tsk["xpro"].capitalize()+" opens the "+tsk["c"]+" and looks inside. "+tsk["xpro"].capitalize()+" calls a friend to tell them that "+tsk["xpro"]+" has just found a "+tsk["c"]+" full of"

    #correct task #17
    if tsk["NUMBER"]==17:
        q1 = tsk["xpro"].capitalize()+" puts one in a CD player and listens to the songs. "+tsk["xpro"].capitalize()+" can clearly hear that it is full of"
        q2 = q2.replace("opening the CD", "listening to the CD")
    if tsk["NUMBER"]==14:
        q2 = q2.replace("found", "received")
    if tsk["NUMBER"]==12:
        q2 = q2.replace("found", "received")
    if tsk["NUMBER"]==1:
        q2 = q2.replace("opening the hard drive", "connecting to the hard drive and exploring its contents")

  if study=="S2":
    txt=tsk[typ]
    if typ=="txt_informed":
        txt=tsk["txt"]+" "+tsk["txt_informed"]
    q1=tsk["q1"]
    q2=tsk["q2"]

  if reverse:
      txt=txt.replace(tsk["o1"], "####").replace(tsk["o2"],tsk["o1"]).replace("####",tsk["o2"])

  out=[]
  for i in [q1,q2]:
      if eng in ["gpt-4", "gpt-3.5-turbo","gpt-4-0314","gpt-3.5-turbo-0301"]:
          out.append(chat(model=eng, msgs=("Complete the following story: "+txt+" "+i).replace("  "," "), max_tokens=len_out,temperature=0)[-1]["content"])
          print("$", end="")
      else: out.append(query(engine=eng, prompt=(txt+" "+i).replace("  "," "), max_tokens=len_out,temperature=0, logprobs=0))

  xout=[re.sub(r'[.,\'"]', '', i.split()[0]) for i in out]

  expected=[tsk["o1"],tsk["o2"]]
  if reverse: expected=[tsk["o2"], tsk["o1"]]
  if typ in ["txt_transparent", "txt_correctlabel", "txt_informed","txt_open"]:
      expected=[tsk["o1"],tsk["o1"]]
      if reverse: expected=[tsk["o2"], tsk["o2"]]
  if typ in ["txt_notransfer"]:
      expected=[tsk["o2"],tsk["o2"]]
      if reverse: expected=[tsk["o1"], tsk["o1"]]
  if typ in ["txt_stayed", "txt_same"]:
     expected=[tsk["o1"],tsk["o1"]]
     if reverse: expected=[tsk["o2"], tsk["o2"]]

  expected=[i.split()[0] for i in expected]
  expected=[i.replace("videos","video") for i in expected]

  if expected==xout: return(True)
  else:
      if not reverse: print("INCORRECT RESPONSE FOR TASK #"+str(tsk["NUMBER"])+ " == "+eng+" "+typ)
      else: print("INCORRECT RESPONSE FOR TASK #"+str(tsk["NUMBER"]+20)+ " == "+eng+" "+typ)
      print("TASK: "+txt+"\n")
      if expected[0]!=xout[0]:   print("Prompt 1: "+q1+"\n###OBS:"+out[0]+"\n###EXP:"+expected[0]+"\n")
      if expected[1]!=xout[1]:   print("Prompt 2: "+q2+"\n###OBS:"+out[1]+"\n###EXP:"+expected[1]+"\n")
      return(False)


In [ ]:
# unexpected Contents tasks
tsks1 = [{'txt': 'On the shelf, there is a non-transparent CX. It is full of S1; there is no S2 in it. Yet, the label on this CX says "S2" and not "S1". XNAM walks into the room and notices the CX. XPRO has never seen it before. XPRO reads the label. XPRO cannot see what is inside.',
  'o1': 'beer',
  'o2': 'wine',
  'c': 'bottle',
  'xnam': 'Mark',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 0,
  'txt_correctlabel': 'On the shelf, there is a non-transparent CX. It is full of S1; there is no S2 in it. The label on this CX says "S1" and not "S2". XNAM walks into the room and notices the CX. XPRO has never seen it before. XPRO reads the label. XPRO cannot see what is inside.',
  'txt_informed': 'On the shelf, there is a non-transparent CX. It is full of S1; there is no S2 in it. Yet, the label on this CX says "S2" and not "S1". XNAM walks into the room and notices the CX. XPRO has never seen it before. XPRO reads the label. XPRO cannot see what is inside. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'On the shelf at the company\'s headquarters, there is a CX that contains only S1 files and no S2 files. Yet, its label clearly states "S2 files" and not "S1 files". XNAM, the newly hired computer engineer, finds the CX on the shelf. XPRO has never seen this CX before. XPRO reads its label.',
  'o1': 'audio',
  'o2': 'video',
  'c': 'hard drive',
  'xnam': 'Sarah',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 1,
  'txt_correctlabel': 'On the shelf at the company\'s headquarters, there is a CX that contains only S1 files and no S2 files. Its label clearly states "S1 files" and not "S2 files". XNAM, the newly hired computer engineer, finds the CX on the shelf. XPRO has never seen this CX before. XPRO reads its label.',
  'txt_informed': 'On the shelf at the company\'s headquarters, there is a CX that contains only S1 files and no S2 files. Yet, its label clearly states "S2 files" and not "S1 files". XNAM, the newly hired computer engineer, finds the CX on the shelf. XPRO has never seen this CX before. XPRO reads its label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'In the freezer, there is a CX filled with S1. There is no S2 in it. Yet, the label says "S2" and not "S1". The label is wrong. One day, XNAM finds the CX and realizes that XPRO has never seen it before. XPRO cannot see what is inside the CX. XPRO reads the label.',
  'o1': 'vanilla ice cream',
  'o2': 'strawberry jam',
  'c': 'container',
  'xnam': 'Floriane',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 2,
  'txt_correctlabel': 'In the freezer, there is a CX filled with S1. There is no S2 in it. The label says "S1" and not "S2". The label is correct. One day, XNAM finds the CX and realizes that XPRO has never seen it before. XPRO cannot see what is inside the CX. XPRO reads the label.',
  'txt_informed': 'In the freezer, there is a CX filled with S1. There is no S2 in it. Yet, the label says "S2" and not "S1". The label is wrong. One day, XNAM finds the CX and realizes that XPRO has never seen it before. XPRO cannot see what is inside the CX. XPRO reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'A CX full of S1 arrives at the port of Amsterdam. There are no S2 in it. It is sealed. Yet, the shipping documents state that it contains S2 and do not mention S1. XNAM, a customs officer, approaches the CX. XPRO cannot see what is inside the CX. XPRO reads the shipping documents to figure out what is inside the CX.',
  'o1': 'computers',
  'o2': 'motorbikes',
  'c': 'shipping container',
  'xnam': 'Mary',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 3,
  'txt_correctlabel': 'A CX full of S1 arrives at the port of Amsterdam. There are no S2 in it. It is sealed. The shipping documents state that it contains S1 and do not mention S2. XNAM, a customs officer, approaches the CX. XPRO cannot see what is inside the CX. XPRO reads the shipping documents to figure out what is inside the CX.',
  'txt_informed': 'A CX full of S1 arrives at the port of Amsterdam. There are no S2 in it. It is sealed. Yet, the shipping documents state that it contains S2 and do not mention S1. XNAM, a customs officer, approaches the CX. XPRO cannot see what is inside the CX. XPRO reads the shipping documents to figure out what is inside the CX. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the shipping documents that say “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'There is a closed CX on the table in an empty meeting room, in which a seminar is about to take place. The CX is filled with S1, and there are no S2 in it. Yet, it is labeled with the word "S2" and not "S1". The first attendee walks into the room and sits in a chair in front of the CX. XPRO reads the label.',
  'o1': 'pens',
  'o2': 'pencils',
  'c': 'box',
  'xnam': 'Attendee',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 4,
  'txt_correctlabel': 'There is a closed CX on the table in an empty meeting room, in which a seminar is about to take place. The CX is filled with S1, and there are no S2 in it. It is labeled with the word "S1" and not "S2". The first attendee walks into the room and sits in a chair in front of the CX. XPRO reads the label.',
  'txt_informed': 'There is a closed CX on the table in an empty meeting room, in which a seminar is about to take place. The CX is filled with S1, and there are no S2 in it. Yet, it is labeled with the word "S2" and not "S1". The first attendee walks into the room and sits in a chair in front of the CX. XPRO reads the label. A friend calls the XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. The XNAM believes POS_PRO friend.'},
 {'txt': "Inside the fridge of the Airbnb host's kitchen is a CX labeled as S2. However, the CX actually contains S1 and no S2. The label does not mention S1. XNAM, a guest who just arrived at the house, opens the fridge and sees the CX. XPRO cannot see what is inside the CX, but XPRO reads the label.",
  'o1': 'pickles',
  'o2': 'tuna',
  'c': 'can',
  'xnam': 'Jill',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 5,
  'txt_correctlabel': 'Inside the fridge of the Airbnb host\'s kitchen is a CX labeled as "S1". The CX contains S1 and no S2. The label does not mention S2. XNAM, a guest who just arrived at the house, opens the fridge and sees the CX. XPRO cannot see what is inside the CX, but XPRO reads the label.',
  'txt_informed': "Inside the fridge of the Airbnb host's kitchen is a CX labeled as S2. However, the CX actually contains S1 and no S2. The label does not mention S1. XNAM, a guest who just arrived at the house, opens the fridge and sees the CX. XPRO cannot see what is inside the CX, but XPRO reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend."},
 {'txt': 'There is an CX lying in front of Julia\'s door. The envelope is filled with S1, and there are no S2 in it. Yet, a label with the word "S2" is stuck on the outside. The label does not mention "S1". XNAM who is leaving POS_PRO apartment for the first time that morning, walks past and notices the CX on the floor. XPRO wonders what is inside the CX and reads the label.',
  'o1': 'leaflets',
  'o2': 'receipts',
  'c': 'envelope',
  'xnam': "Julia's neighbor",
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 6,
  'txt_correctlabel': 'There is an CX lying in front of Julia\'s door. The envelope is filled with S1, and there are no S2 in it. A label with the word "S1" is stuck on the outside. The label does not mention "S2". XNAM who is leaving POS_PRO apartment for the first time that morning, walks past and notices the CX on the floor. XPRO wonders what is inside the CX and reads the label.',
  'txt_informed': 'There is an CX lying in front of Julia\'s door. The envelope is filled with S1, and there are no S2 in it. Yet, a label with the word "S2" is stuck on the outside. The label does not mention "S1". XNAM who is leaving POS_PRO apartment for the first time that morning, walks past and notices the CX on the floor. XPRO wonders what is inside the CX and reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'A vault that\'s been unopened for over 100 years has just been unlocked. Inside lies a closed CX with an engraving on it that reads "S2". It does not mention "S1". However, the CX actually contains only S1 and doesn\'t contain any S2 at all. XNAM, an inspector, approaches the vault. XPRO cannot see what is inside the CX. XPRO reads the engraving.',
  'o1': 'gems',
  'o2': 'books',
  'c': 'box',
  'xnam': 'Steve',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 7,
  'txt_correctlabel': 'A vault that\'s been unopened for over 100 years has just been unlocked. Inside lies a closed CX with an engraving on it that reads "S1". It does not mention "S2". The CX contains only S1 and doesn\'t contain any S2 at all. XNAM, an inspector, approaches the vault. XPRO cannot see what is inside the CX. XPRO reads the engraving.',
  'txt_informed': 'A vault that\'s been unopened for over 100 years has just been unlocked. Inside lies a closed CX with an engraving on it that reads "S2". It does not mention "S1". However, the CX actually contains only S1 and doesn\'t contain any S2 at all. XNAM, an inspector, approaches the vault. XPRO cannot see what is inside the CX. XPRO reads the engraving. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the engraving that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'A non-transparent CX filled with S1 sits on the counter of a department store. There is no S2 inside it; however, the CX bears the label "S2" instead of "S1". Bernard, a new customer, walks up to the counter and notices the CX. XPRO reads the label.',
  'o1': 'sugar',
  'o2': 'water',
  'c': 'bottle',
  'xnam': 'Bernard',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 8,
  'txt_correctlabel': 'A non-transparent CX filled with S1 sits on the counter of a department store. There is no S2 inside it; the CX bears the label "S1" and not "S2". Bernard, a new customer, walks up to the counter and notices the CX. XPRO reads the label.',
  'txt_informed': 'A non-transparent CX filled with S1 sits on the counter of a department store. There is no S2 inside it; however, the CX bears the label "S2" instead of "S1". Bernard, a new customer, walks up to the counter and notices the CX. XPRO reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': "In an abandoned mansion, a forensics team is inspecting a crime scene. Next to the body is a closed CX; no one can see what is inside. Although the CX actually contains S1, its label says S2. The XNAM, who's just arrived on the scene, crouches down to inspect the CX. XPRO reads the label. ",
  'o1': 'pills',
  'o2': 'bullets',
  'c': 'box',
  'xnam': 'lead detective',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 9,
  'txt_correctlabel': "In an abandoned mansion, a forensics team is inspecting a crime scene. Next to the body is a closed CX; no one can see what is inside. The CX actually contains S1 and its label says so. There is no S2 in it. The XNAM, who's just arrived on the scene, crouches down to inspect the CX. XPRO reads the label. ",
  'txt_informed': "In an abandoned mansion, a forensics team is inspecting a crime scene. Next to the body is a closed CX; no one can see what is inside. Although the CX actually contains S1, its label says S2. The XNAM, who's just arrived on the scene, crouches down to inspect the CX. XPRO reads the label. A friend calls the XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. The XNAM believes POS_PRO friend. "},
 {'txt': 'Julia prepares a CX for her son, XNAM, under the Christmas tree. She wants to prank Steven. She fills the CX with S1 but labels it S2, written in clear large letters. A few hours later, XNAM comes down the stairs and finds the CX under the Christmas tree. He cannot see what is inside, but XPRO reads the label.',
  'o1': 'tea leaves',
  'o2': 'coffee beans',
  'c': 'gift box',
  'xnam': 'Steven',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 10,
  'txt_correctlabel': 'Julia prepares a CX for her son, XNAM, under the Christmas tree. She wants to prank Steven. She fills the CX with S1 and labels it S1, written in clear large letters. A few hours later, XNAM comes down the stairs and finds the CX under the Christmas tree. He cannot see what is inside, but XPRO reads the label.',
  'txt_informed': 'Julia prepares a CX for her son, XNAM, under the Christmas tree. She wants to prank Steven. She fills the CX with S1 but labels it S2, written in clear large letters. A few hours later, XNAM comes down the stairs and finds the CX under the Christmas tree. He cannot see what is inside, but XPRO reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'At the customs department of an airport, an immigration officer hands a CX to a biohazard inspector. It is sealed and labeled as "S2 samples". No one can see what is inside, but in reality, it only contains S1 samples. The inspector has never examined the contents of this CX before. XPRO reads the label on the CX.',
  'o1': 'blood',
  'o2': 'urine',
  'c': 'box',
  'xnam': 'inspector',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 11,
  'txt_correctlabel': 'At the customs department of an airport, an immigration officer hands a CX to a biohazard inspector. It is sealed and labeled as "S1 samples". No one can see what is inside, but in reality, there are no S2 samples in it, only S1 samples. The inspector has never examined the contents of this CX before. XPRO reads the label on the CX.',
  'txt_informed': 'At the customs department of an airport, an immigration officer hands a CX to a biohazard inspector. It is sealed and labeled as "S2 samples". No one can see what is inside, but in reality, it only contains S1 samples. The inspector has never examined the contents of this CX before. XPRO reads the label on the CX. A friend calls the XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. The XNAM believes POS_PRO friend.'},
 {'txt': 'Paul holds out a gift-wrapped CX to XNAM and tells POS_PRO that there are S2 in it. In reality, though, the CX contains only S1. XNAM takes the CX from him.',
  'o1': 'socks',
  'o2': 'scarves',
  'c': 'box',
  'xnam': 'Sara',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 12,
  'txt_correctlabel': 'Paul holds out a gift-wrapped CX to XNAM and tells POS_PRO that there are S1 in it. The CX contains only S1 and no S2. XNAM takes the CX from him.',
  'txt_informed': 'Paul holds out a gift-wrapped CX to XNAM and tells POS_PRO that there are S2 in it. In reality, though, the CX contains only S1. XNAM takes the CX from him. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore that Paul tells her that it contains “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'A deceased person\'s laptop contains an encrypted CX labeled "S2". However, in reality, the CX only contains S1. A digital forensics expert has been hired to retrieve and back up the contents of this CX. As XPRO switches on the machine for the first time, the home screen appears. She locates the encrypted CX and reads its label.',
  'o1': 'videos',
  'o2': 'photos',
  'c': 'folder',
  'xnam': 'forensics expert',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 13,
  'txt_correctlabel': 'A deceased person\'s laptop contains an encrypted CX labeled "S1". The CX only contains S1. A digital forensics expert has been hired to retrieve and back up the contents of this CX. As XPRO switches on the machine for the first time, the home screen appears. She locates the encrypted CX and reads its label.',
  'txt_informed': 'A deceased person\'s laptop contains an encrypted CX labeled "S2". However, in reality, the CX only contains S1. A digital forensics expert has been hired to retrieve and back up the contents of this CX. As XPRO switches on the machine for the first time, the home screen appears. She locates the encrypted CX and reads its label. A friend calls the XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. The XNAM believes POS_PRO friend.'},
 {'txt': 'On Thursday, XNAM orders some S2 on the internet. There is a mistake at the logistics center. They ship a CX full of S1 but label it as "S2". The label does not mention "S1" at all. The CX arrives on Saturday morning. The postman leaves the CX at XNAM\'s front door and rings the doorbell. XNAM opens the door, looks down, and sees the CX. XPRO reads the label. He does not open the CX.',
  'o1': 'wallets',
  'o2': 'sneakers',
  'c': 'parcel',
  'xnam': 'Daniel',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 14,
  'txt_correctlabel': 'On Thursday, XNAM orders some S2 on the internet. Unfortunately, there is a mistake at the logistics center. They ship a CX full of S1 and label it as "S1". The label does not mention "S2" at all. The CX arrives on Saturday in the morning. The postman leaves the CX at XNAM\'s front door and rings the doorbell. XNAM opens the door, looks down, and sees the CX. XPRO reads the label. He does not open the CX.',
  'txt_informed': 'On Thursday, XNAM orders some S2 on the internet. Unfortunately, there is a mistake at the logistics center. They ship a CX full of S1 but label it as "S2". The label does not mention "S1" at all. The CX arrives on Saturday in the morning. The postman leaves the CX at XNAM\'s front door and rings the doorbell. XNAM opens the door, looks down, and sees the CX. XPRO reads the label. He does not open the CX. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'A CX has been left behind at the park after a school picnic day event. The CX has only S1 in it, but a sticker on the outside says "S2". One of the organizers sees the CX as he strolls past the bench where it\'s sitting. XPRO has never seen it before. XPRO reads the sticker.',
  'o1': 'sandwiches',
  'o2': 'beer',
  'c': 'cooler box',
  'xnam': 'organizer',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 15,
  'txt_correctlabel': 'A CX has been left behind at the park after a school picnic day event. The CX has only S1 in it, and no S2. Also, the sticker on the outside says "S1". One of the organizers sees the CX as he strolls past the bench where it\'s sitting. XPRO has never seen it before. XPRO reads the sticker.',
  'txt_informed': 'A CX has been left behind at the park after a school picnic day event. The CX has only S1 in it, but a sticker on the outside says "S2". One of the organizers sees the CX as he strolls past the bench where it\'s sitting. XPRO has never seen it before. XPRO reads the sticker. A friend calls the XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the sticker that says “S2”. The XNAM believes POS_PRO friend.'},
 {'txt': 'The packers at a freight forwarding company are notified of a CX that has just arrived at their headquarters. The bill of lading lists the contents of this CX as S2 and does not mention S1. One of the packers, XNAM, is sent to collect the CX. However, no one informs him that the CX actually contains S1 and there are no S2 in it. XNAM reads the bill of lading to understand what is inside the CX.',
  'o1': 'clothes',
  'o2': 'vegetables',
  'c': 'container',
  'xnam': 'Daniel',
  'xpro': 'he',
  'obj_pro': 'him',
  'pos_pro': 'his',
  'NUMBER': 16,
  'txt_correctlabel': 'The packers at a freight forwarding company are notified of a CX that has just arrived at their headquarters. The bill of lading lists the contents of this CX as S1 and does not mention S2. One of the packers, XNAM, is sent to collect the CX. However, no one informs him that the CX contains S1 and there are no S2 in it. XNAM reads the bill of lading to understand what is inside the CX.',
  'txt_informed': 'The packers at a freight forwarding company are notified of a CX that has just arrived at their headquarters. The bill of lading lists the contents of this CX as S2 and does not mention S1. One of the packers, XNAM, is sent to collect the CX. However, no one informs him that the CX actually contains S1 and there are no S2 in it. XNAM reads the bill of lading to understand what is inside the CX. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the bill of lading that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'In the attic of XNAM\'s mother, there is a box of CDs labeled as "S2". The label is incorrect because the CDs only contain S1 music. XNAM is rummaging around the attic and finds this box. XPRO has never seen or listened to these CDs before. XPRO reads the label.',
  'o1': 'jazz',
  'o2': 'disco',
  'c': 'CD',
  'xnam': 'Vicky',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 17,
  'txt_correctlabel': 'In the attic of XNAM\'s mother, there is a box of CDs labeled as "S1". The label is correct because the CDs only contain S1 music and no S2 music. XNAM is rummaging around the attic and finds this box. XPRO has never seen or listened to these CDs before. XPRO reads the label.',
  'txt_informed': 'In the attic of XNAM\'s mother, there is a box of CDs labeled as "S2". The label is incorrect because the CDs only contain S1 music. XNAM is rummaging around the attic and finds this box. XPRO has never seen or listened to these CDs before. XPRO reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'In a bathroom of a hotel room there is a metal CX, standing next to the sink. It contains S1 but the lettering on its lid says "S2". XNAM has just checked into this hotel room and goes to the bathroom. XNAM spots the CX and glances at the lettering on its lid.',
  'o1': 'soap',
  'o2': 'cream',
  'c': 'box',
  'xnam': 'Megan',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 18,
  'txt_correctlabel': 'In a bathroom of a hotel room there is a metal CX, standing next to the sink. It contains S1 and the lettering on its lid says "S1". There is no S2 in it. XNAM has just checked into this hotel room and goes to the bathroom. XNAM spots the CX and glances at the lettering on its lid.',
  'txt_informed': 'In a bathroom of a hotel room there is a metal CX, standing next to the sink. It contains S1 but the lettering on its lid says "S2". XNAM has just checked into this hotel room and goes to the bathroom. XNAM spots the CX and glances at the lettering on its lid. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the lettering that says “S2”. XNAM believes POS_PRO friend.'},
 {'txt': 'Here is a CX filled with S1. There is no S2 in this CX. Yet, the label on this CX says "S2" and not "S1". XNAM finds the CX. XPRO has never ever seen this CX before. Also, XPRO cannot see what is inside this CX. XPRO reads the label.',
  'o1': 'chocolate',
  'o2': 'popcorn',
  'c': 'bag',
  'xnam': 'Sam',
  'xpro': 'she',
  'obj_pro': 'her',
  'pos_pro': 'her',
  'NUMBER': 19,
  'txt_correctlabel': 'Here is a CX filled with S1. There is no S2 in this CX. Yet, the label on this CX says "S1" and not "S2". XNAM finds the CX. XPRO has never ever seen this CX before. Also, XPRO cannot see what is inside this CX. XPRO reads the label.',
  'txt_informed': 'Here is a CX filled with S1. There is no S2 in this CX. Yet, the label on this CX says "S2" and not "S1". XNAM finds the CX. XPRO has never ever seen this CX before. Also, XPRO cannot see what is inside this CX. XPRO reads the label. A friend calls XNAM and tells OBJ_PRO that in reality, the CX has S1 in it, and that XPRO should ignore the label that says “S2”. XNAM believes POS_PRO friend.'}]

## Sanity check
process_tsk(tsks1[0], typ="txt", len_out=1, eng="gpt-3.5-turbo", study="S1", reverse=False)

INPUT: Complete the following story: On the shelf, there is a non-transparent bottle. It is full of beer; there is no wine in it. Yet, the label on this bottle says "wine" and not "beer". Mark walks into the room and notices the bottle. He has never seen it before. He reads the label. He cannot see what is inside. He opens the bottle and looks inside. He can clearly see that it is full of
gpt-3.5-turbo: beer
$INPUT: Complete the following story: On the shelf, there is a non-transparent bottle. It is full of beer; there is no wine in it. Yet, the label on this bottle says "wine" and not "beer". Mark walks into the room and notices the bottle. He has never seen it before. He reads the label. He cannot see what is inside. Before opening the bottle, he calls a friend to tell them that he has just found a bottle full of
gpt-3.5-turbo: wine
$

True

In [ ]:
### Consider using a loop to process tasks for different engines
e="gpt-3.5-turbo"
n=10

### false-belief
txt=[process_tsk(i, eng=e, typ="txt", len_out=n, study="S1",reverse=False) for i in tsks1] ### use reverse=True for reversed tasks
### correct label
txt_correctlabel=[process_tsk(i, eng=e, typ="txt_correctlabel", len_out=n, study="S1",reverse=False) for i in tsks1] ### use reverse=True for reversed tasks
### informed protagonist
txt_informed=[process_tsk(i, eng=e, typ="txt_informed", len_out=n, study="S1",reverse=False) for i in tsks1] ### use reverse=True for reversed tasks
### open container
txt_open=[process_tsk(i, eng=e, typ="txt_open", len_out=n, study="S1",reverse=False) for i in tsks1] ### use reverse=True for reversed tasks

In [ ]:
# unexpected transfer tasks
tsks2 = [{'txt': 'Father comes home from the grocery store. He bought bananas for banana bread. His daughter, Maria, helps him put away the shopping. She asks him: "Where should I put the bananas?" "In the red drawer",says Father. "Wait, I\'ll lift you up there, because you are too short", says Father, and lifts her up. Maria puts the bananas in the red drawer. She loves bananas. Then, she leaves for school. While Maria is away, Father starts to prepare the banana bread and takes the bananas from the red drawer. He peels and slices one of them, but then he puts the rest of them back in the yellow drawer instead of the red one. He realizes that he is missing eggs. He leaves the kitchen and goes to see if their neighbor has any eggs. Maria comes back to the kitchen and wants to eat a banana.',
  'o1': 'yellow',
  'o2': 'red',
  'q1': 'The bananas fall out of the',
  'q2': 'Maria will look for the bananas in the',
  'NUMBER': 0,
  'txt_notransfer': 'Father comes home from the grocery store. He bought bananas for banana bread. His daughter, Maria, helps him put away the shopping. She asks him: "Where should I put the bananas?" "In the red drawer",says Father. "Wait, I\'ll lift you up there, because you are too short", says Father, and lifts her up. Maria puts the bananas in the red drawer. She loves bananas. Then, she leaves for school. While Maria is away, Father starts to prepare the banana bread and takes the bananas from the red drawer. He peels and slices one of them, but then he puts the rest of them back in the red drawer. He realizes that he is missing eggs. He leaves the kitchen and goes to see if their neighbor has any eggs. Maria comes back to the kitchen and wants to eat a banana.',
  'txt_informed': 'Father calls Maria and tells her that he moved the bananas from the red drawer to the yellow drawer. Maria believes her Father.',
  'txt_same': 'Father comes home from the grocery store. He bought bananas for banana bread. His daughter, Maria, helps him put away the shopping. She asks him: "Where should I put the bananas?" "In the red drawer", says Father. "Wait, I\'ll lift you up there, because you are too short", says Father, and lifts her up. Maria puts the bananas in the red drawer. She loves bananas. Father starts to prepare the banana bread and takes the bananas from the red drawer. He peels and slices one of them. Maria puts the rest of them back in the yellow drawer instead of the red one. Her father realizes that he is missing eggs. He leaves the kitchen and goes to see if their neighbor has any eggs. Maria wants to eat a banana.'},
 {'txt': "Cecilia is about to bake a saffron cake for her son's birthday. In preparation, she places a small packet of very expensive saffron in the drawer, which is under the cupboard. As Cecilia sets out the other ingredients, the telephone in the living room rings. She leaves the kitchen to attend the call. She closes the door behind her and cannot see what is happening in the kitchen. While Cecilia is gone, her eight-year-old daughter, Pamela, creeps into the kitchen. Pamela is jealous about all the attention that her brother will be receiving on his birthday, so she quietly takes the packet of saffron from the drawer and hides it in the cupboard. She leaves the kitchen and goes to school. Cecilia comes back to the kitchen and wants to use some saffron. ",
  'o1': 'cupboard',
  'o2': 'drawer',
  'q1': 'The packet of saffron falls out of the',
  'q2': 'Cecilia will look for the saffron in the',
  'NUMBER': 1,
  'txt_notransfer': "Cecilia is about to bake a saffron cake for her son's birthday. In preparation, she places a small packet of very expensive saffron in the drawer, which is under the cupboard. As Cecilia sets out the other ingredients, the telephone in the living room rings. She leaves the kitchen to attend the call. She closes the door behind her and cannot see what is happening in the kitchen. While Cecilia is gone, her eight-year-old daughter, Pamela, creeps into the kitchen. Pamela is jealous about all the attention that her brother will be receiving on his birthday, so she quietly looks at the packet of saffron in the drawer but leaves it there. She leaves the kitchen and goes to school. Cecilia comes back to the kitchen and wants to use some saffron.",
  'txt_informed': 'Pamela calls Cecilia and tells her that she moved the packet of saffron from the drawer to the cupboard. Cecilia believes Pamela.',
  'txt_same': "Cecilia is about to bake a saffron cake for her son's birthday. In preparation, she places a small packet of very expensive saffron in the drawer, which is under the cupboard. As Cecilia sets out the other ingredients, her eight-year-old daughter, Pamela, walks into the kitchen. Pamela is jealous about all the attention that her brother will be receiving on his birthday. Cecilia takes the packet of saffron from the drawer and puts it in the cupboard. Pamela leaves the kitchen and goes to school. Cecilia wants to use some saffron. "},
 {'txt': "It's the end of the day shift at a newsroom, and a reporter has almost finished writing an article on the office computer. There are two folders on the desktop screen: Drafts and Edits. Intending to complete the article the next day, she saves the article in Drafts and leaves the newsroom. Soon she is at home and fast asleep. When the reporter is gone, the staff proofreader enters the office to begin his night shift. He browses through the two folders to see if there's anything he should work on. He notices that the article that the reporter wrote is an urgent piece that needs to be published as soon as possible. He decides to write the concluding paragraph and have it ready for the design team. After the article is finished and proofread, he moves it into the Edits folder. He goes home. The next day in the morning, the reporter wakes up. She comes back to the newsroom and wants to continue working on the article.",
  'o1': 'Edits',
  'o2': 'Drafts',
  'q1': 'The article file is in the folder labeled',
  'q2': 'The reporter will look for the article file in the folder labeled',
  'NUMBER': 2,
  'txt_notransfer': "It's the end of the day shift at a newsroom, and a reporter has almost finished writing an article on the office computer. There are two folders on the desktop screen: Drafts and Edits. Intending to complete the article the next day, she saves the article in Drafts and leaves the newsroom. Soon she is at home and fast asleep. When the reporter is gone, the staff proofreader enters the office to begin his night shift. He browses through the two folders to see if there's anything he should work on. He notices that the article that the reporter wrote is an urgent piece that needs to be published as soon as possible. He decides to write the concluding paragraph and have it ready for the design team. After the article is finished and proofread, he leaves the file in the Drafts folder. He goes home. The next day in the morning, the reporter wakes up. She comes back to the newsroom and wants to continue working on the article.",
  'txt_informed': 'The proofreader calls the reporter and tells her that he moved the article from the Drafts folder to the Edits folder. The reporter believes the proofreader.',
  'txt_same': "It's the end of the day shift at a newsroom, and a reporter has almost finished writing an article on the office computer. There are two folders on the desktop screen: Drafts and Edits. Intending to complete the article the next day, she saves the article in Drafts and leaves the newsroom. The staff proofreader enters the office to begin his night shift. He browses through the two folders to see if there's anything he should work on. He notices that the article that the reporter wrote is an urgent piece that needs to be published as soon as possible. The reporter decides to write the concluding paragraph and have it ready for the design team. After the article is finished and proofread, the reporter moves it into the Edits folder. The next day in the morning, the reporter wakes up. She comes back to the newsroom and wants to continue working on the article."},
 {'txt': "Lila likes it when her dog stays in the house while she's away. Thus, she puts her dog in the house before leaving for school. When Lila is gone, her mom comes home. Mom doesn't like it when the dog is locked in the house, so she takes the dog outside and puts it in the shed instead. She locks the shed's door and leaves to work. Lila comes back from school and wants to play with the dog. ",
  'o1': 'shed',
  'o2': 'house',
  'q1': 'The dog escapes from the',
  'q2': 'Lila will look for the dog in the',
  'NUMBER': 3,
  'txt_notransfer': "Lila likes it when her dog stays in the house while she's away. Thus, she puts her dog in the house before leaving for school. When Lila is gone, her mom comes home. Mom doesn't like it when the dog is locked in the house, so she takes the dog outside. After a few moments she puts it back in the house, locks the door and leaves to work. Lila comes back from school and wants to play with the dog.",
  'txt_informed': "Lila's mom calls Lila and tells her that she moved the dog from the house to the shed. Lila believes her mom.",
  'txt_same': "Lila likes it when her dog stays in the house. Thus, she puts her dog in the house. Her mom comes home; she doesn't like it when the dog is locked in the house. Lila takes the dog outside and puts it in the shed instead. She locks the shed's door. Now, Lila wants to play with the dog. "},
 {'txt': "It's nighttime, and Steven is doing laundry in the laundry room. After washing a blanket, he puts it in the dryer and then goes to bed. Later that night, Steven's sister comes home and walks into the laundry room. She takes the blanket out of the dryer and puts it in the cupboard next to the dryer. She goes to sleep. Steven wakes up in the morning. He goes to the laundry room and wants to get the blanket. ",
  'o1': 'cupboard',
  'o2': 'dryer',
  'q1': 'The blanket falls out from the',
  'q2': 'He will look for the blanket in the',
  'NUMBER': 4,
  'txt_notransfer': "It's nighttime, and Steven is doing laundry in the laundry room. After washing a blanket, he puts it in the dryer and then goes to bed. Later that night, Steven's sister comes home and walks into the laundry room. She sees the blanket in the dryer but leaves it there and puts nothing in the cupboard next to the dryer. She goes to sleep. Steven wakes up in the morning. He goes to the laundry room and wants to get the blanket.",
  'txt_informed': "Steven's sister calls Steven and tells him that she moved the blanket from the dryer to the cupboard. Steven believes his sister.",
  'txt_same': "It's nighttime, and Steven is doing laundry in the laundry room. After washing a blanket, he puts it in the dryer. He watches as his sister walks into the laundry room. Now, Steven takes the blanket out of the dryer, and puts it in the cupboard next to the dryer. His sister goes to sleep. Now, Steven wants to get the blanket. "},
 {'txt': 'Monica and Floriane are in a room. Monica puts a toy in a box. She closes the box. Monica leaves the room and goes outside to play with other children. When Monica is gone, Floriane removes the toy from the box and puts it in a basket. She closes the basket and leaves for school. Monica comes back to the room and wants to play with the toy.',
  'o1': 'basket',
  'o2': 'box',
  'q1': 'The toy falls out of the',
  'q2': 'Monica will look for the toy in the',
  'NUMBER': 5,
  'txt_notransfer': 'Monica and Floriane are in a room. Monica puts a toy in a box. She closes the box. Monica leaves the room and goes outside to play with other children. When Monica is gone, Floriane looks at the toy in the box but leaves it there. She closes the box and leaves for school. Monica comes back to the room and wants to play with the toy.',
  'txt_informed': 'Monica calls Floriane and tells her that she moved the toy from the box to the basket. Floriane believes Monica.',
  'txt_same': 'Monica and Floriane are in a room. Monica puts a toy in a box. She closes the box. Monica removes the toy from the box and puts it in a basket. Floriane closes the basket and leaves for school. Monica wants to play with the toy.'},
 {'txt': "Mark puts his blue basketball in a wooden box and closes it. Mark doesn't know it, but this is a magical box, which changes colors of things that are left inside. After a few minutes, the basketball turns green. Mark looks at the box but cannot see what is happening inside.",
  'o1': 'green',
  'o2': 'blue',
  'q1': 'Mark opens the box and can see that the the color of the basketball is',
  'q2': 'Before opening the box, Mark calls his friend to tell them that the color of the basketball is',
  'NUMBER': 6,
  'txt_notransfer': "Mark puts his blue basketball in a wooden box and closes it. Mark doesn't know it, but this is a magical box, which changes colors of things that are left inside. After a few minutes, the basketball turns green, but then turns blue again. Mark looks at the box but cannot see what is happening inside. Mary calls Mark and asks him what is the color of his ball. Mark tells her the truth.",
  'txt_informed': 'The magical box tells Mark that it changed the color of the basketball from blue to green. Mark believes the box.',
  'txt_same': 'Mark puts his blue basketball in his magical wooden box and closes it. Mark knows that this box changes colors. Anything that is put inside turns green. As Mark is watching, the basketball turns green. Mary calls Mark and asks him what is the color of his ball. Mark tells her the truth.'},
 {'txt': 'Max watches his little brother put laundry in the box. Next, Max leaves the house to play in the courtyard. While Max is gone, his brother moves the laundry from the box to the basket. He closes the basket and leaves for school. Max comes back home and wants to fold the laundry.',
  'o1': 'basket',
  'o2': 'box',
  'q1': 'The laundry falls out of the',
  'q2': 'Max will look for the laundry in the',
  'NUMBER': 7,
  'txt_notransfer': 'Max watches his little brother put laundry in the box. Next, Max leaves the house to play in the courtyard. While Max is gone, his brother thinks about moving the laundry from the box to the basket, but decides against it. He closes the basket and leaves for school. Max comes back home and wants to fold the laundry.',
  'txt_informed': "Max's brother calls Max and tells him that he moved the laundry from the box to the basket. Max believes his brother.",
  'txt_same': "Max watches his little brother put laundry in the box. Next, he  moves the laundry from the box to the basket. Max's brother closes the basket and leaves for school. Max wants to fold the laundry."},
 {'txt': 'Ed and his sister, Lilly, are in a room. They have two toys: a truck and a robot. They play with these toys until Ed accidentally breaks the truck. Disappointed, he leaves the room to look for his mom. In his absence, Lilly repairs the truck, but inadvertently breaks the robot. Subsequently, she too leaves the room. Ed finds his mom.',
  'o1': 'robot',
  'o2': 'truck',
  'q1': 'The toy that needs to be repaired is the',
  'q2': 'Ed will tell his mom that they need to repair the',
  'NUMBER': 8,
  'txt_notransfer': 'Ed and his sister, Lilly, are in a room. They have two toys: a truck and a robot. They play with these toys until Ed accidentally breaks the truck. Disappointed, he leaves the room to look for his mom. In his absence, Lilly repairs the truck, but inadvertently breaks it again. Subsequently, she too leaves the room. Ed finds his mom.',
  'txt_informed': "Ed's sister calls Ed and tells him that she repaired the truck, but inadvertently broke the robot. Ed believes his sister.",
  'txt_same': 'Ed and his sister, Lilly, are in a room. They have two toys: a truck and a robot. They play with these toys until Ed accidentally breaks the truck. Next, Ed repairs the truck, but inadvertently breaks the robot. His sister leaves the room. Ed calls his mom.'},
 {'txt': "Mark sees a packet of cheese in the fridge, which is close to its expiry date. He plans to use it later that night when making dinner. He leaves for work. Soon, Mark's wife Jenna enters the to check which groceries need to be disposed of or restocked. She notices the cheese and places it in the freezer, thinking it will last longer that way. She leaves the kitchen and goes to work. Mark comes back home and wants to eat some cheese. ",
  'o1': 'freezer',
  'o2': 'fridge',
  'q1': 'The cheese falls out of the',
  'q2': 'Mark will look for the cheese in the',
  'NUMBER': 9,
  'txt_notransfer': "Mark sees a packet of cheese in the fridge, which is close to its expiry date. He plans to use it later that night when making dinner. He leaves for work. Soon, Mark's wife Jenna enters the kitchen to check which groceries need to be disposed of or restocked. She notices the cheese but leaves it in the fridge, thinking Mark might use it later. She leaves the kitchen and goes to work. Mark comes back home and wants to eat some cheese.",
  'txt_informed': 'Jenna calls Mark and tells him that she moved the packet of cheese from the fridge to the freezer. Mark believes Jenna.',
  'txt_same': "Mark sees a packet of cheese in the fridge, which is close to its expiry date. He plans to use it later that night when making dinner. Soon, Mark's wife Jenna enters the kitchen. Mark wants to check which groceries need to be disposed of or restocked. Mark notices the cheese and places it in the freezer, thinking it will last longer that way. Jenna leaves the kitchen and goes to work. Mark wants to eat some cheese."},
 {'txt': "At the end of the day, Anita is packing up her belongings as she gets ready to leave her workplace. She forgets her watch in the conference room. After she leaves, Anita's secretary comes in to tidy up. He sees the watch and picks it up. He leaves the watch in the break room. He plans to tell Anita that he found her watch first thing the next morning. On her way home, Anita realizes that she has forgotten her watch. She returns to the office to retrieve it. ",
  'o1': 'break room',
  'o2': 'conference room',
  'q1': 'A thief steals the watch from the',
  'q2': 'Anita will look for her watch in the',
  'NUMBER': 10,
  'txt_notransfer': "At the end of the day, Anita is packing up her belongings as she gets ready to leave her workplace. She forgets her watch in the conference room. After she leaves, Anita's secretary comes in to tidy up. He sees the watch but leaves it where it is. He plans to tell Anita that he found her watch first thing the next morning. On her way home, Anita realizes that she has forgotten her watch. She returns to work to retrieve it.",
  'txt_informed': 'The secretary, John, calls Anita and tells her that he moved her watch from the conference room to the break room. Anita believes John.',
  'txt_same': "At the end of the day, Anita is packing up her belongings as she gets ready to leave her workplace. She puts her watch in the conference room. Anita's secretary comes in to tidy up. Anita picks the watch and puts it in the break room. She plans to pick it up first thing the next morning. On her way home, Anita realizes that she needs her watch. She returns to the office to retrieve it. "},
 {'txt': 'Frank and his wife Maria have two bank accounts: savings and checking. Their savings account was empty, but early in January, Frank sold their old car for $5,000 and deposited the money in their savings account. In February, Maria realized that they owe a lot of taxes. She logged in to their online bank platform and transferred the $5,000 from their savings account to their checking account. Unfortunately, she forgot to tell Frank that she did that. Frank, who does not have access to online banking, is unable to check the status of the accounts. A few days later, Frank goes to the bank to withdraw the money from the sale of their old car.',
  'o1': 'checking',
  'o2': 'savings',
  'q1': 'The proceeds from selling their old car are in their',
  'q2': 'Frank will asks the bank teller to look for the money in their',
  'NUMBER': 11,
  'txt_notransfer': 'Frank and his wife Maria have two bank accounts: savings and checking. Their savings account was empty, but early in January, Frank sold their old car for $5,000 and deposited the money in their savings account. In February, Maria realized that they owe a lot of taxes. She thought about transferring the $5,000 from their savings account to their checking account, but decided against it. Unfortunately, she forgot to tell Frank about her consideration. Frank, who does not have access to online banking, is unable to check the status of the accounts. A few days later, Frank goes to the bank to withdraw the money from the sale of their old car.',
  'txt_informed': 'Maria calls Frank and tells him that she moved the $5,000 from the savings account to the checking account. Maria believes Frank.',
  'txt_same': 'Frank and his wife Maria have two bank accounts: savings and checking. Their savings account was empty, but early in January, Frank sold their old car for $5,000 and deposited the money in their savings account. In February, Maria told Frank that they owe a lot of taxes. Frank logged in to their online bank platform and transferred the $5,000 from their savings account to their checking account. A few days later, Frank goes to the bank to withdraw the money from the sale of their old car.'},
 {'txt': 'A mermaid finds a hermit crab hidden in a blue shell. She is excited to tell other mermaids about her discovery and swims away to look for them. In the meantime, the hermit crab decides that the blue shell is rather small and uncomfortable. He decides to swap it for a green shell that he has found nearby. He brings along all of his belongings and hides in the green shell. The blue shell is now abandoned and empty. An octopus observes all this and ponders whether it is a good time to try to catch and eat the crab, but then decides that it is still too early for dinner. A few nautical miles away, the mermaid finds her friends smoking seaweed. They all go back to see the crab.',
  'o1': 'green',
  'o2': 'blue',
  'q1': 'The crab jumps out of the',
  'q2': 'The mermaid will look for the crab in the',
  'NUMBER': 12,
  'txt_notransfer': 'A mermaid finds a hermit crab hidden in a blue shell. She is excited to tell other mermaids about her discovery and swims away to look for them. In the meantime, the hermit crab considers that the blue shell is rather small and uncomfortable. He contemplates swapping it for a green shell that he has found nearby. However, he decides to stay in the blue shell and brings all of his belongings back inside. The blue shell remains occupied. An octopus observes all this and ponders whether it is a good time to try to catch and eat the crab, but then decides that it is still too early for dinner. A few nautical miles away, the mermaid finds her friends smoking seaweed. They all go back to see the crab.',
  'txt_informed': 'The hermit crab calls the mermaid and tells her that he moved from the blue shell to the green shell. The mermaid believes the hermit crab.',
  'txt_same': 'A mermaid finds a hermit crab hidden in a blue shell. She is excited to tell other mermaids about her discovery. She decides that the blue shell is rather small and uncomfortable and moves the crab to a green shell that she has found nearby. She also moves along all of his belongings. The blue shell is now abandoned and empty. She is excited to tell other mermaids about her discovery and swims away to look for them. An octopus observes all this and ponders whether it is a good time to try to catch and eat the crab,but then decides that it is still too early for dinner. A few nautical miles away, the mermaid finds her friends smoking seaweed. They all go back to see the crab.'},
 {'txt': 'The insurgents hide in the forest in Nottinghamshire. They are planning their next attack on the army checkpoint. One of the local farmers spots the insurgents and reports their whereabouts to the army. The army puts the troops on high alert and prepares to surround the insurgents. The insurgents realize what is going on and decide to abandon their base. They pack their tents and their weapons, and move to the forest in Oxfordshire. They do so in a stealthy way, so no one realizes that they have moved. They set up their base and start planning their next cunning attack. In the meantime, the army gets ready to attack.',
  'o1': 'Oxfordshire',
  'o2': 'Nottinghamshire',
  'q1': 'The insurgents march out of the forest located in',
  'q2': 'The army will look for the insurgents in the forest located in',
  'NUMBER': 13,
  'txt_notransfer': 'The insurgents hide in the forest in Nottinghamshire. They are planning their next attack on the army checkpoint. One of the local farmers spots the insurgents and reports their whereabouts to the army. The army puts the troops on high alert and prepares to surround the insurgents. The insurgents realize what is going on and decide to fortify their base. They pack their tents and their weapons, and prepare to defend the forest in Nottinghamshire. They do so in a stealthy way, so no one realizes that they have stayed. They fortify their base and start planning their next cunning attack. In the meantime, the army gets ready to attack.',
  'txt_informed': 'One of the insurgents, John, calls his contact in the army, Lieutenant Smith, and tells him that they moved their base from Nottinghamshire to Oxfordshire. Lieutenant Smith believes John.',
  'txt_same': 'The insurgents hide in the forest in Nottinghamshire. They are planning their next attack on the army checkpoint. One of the local farmers spots the insurgents and reports their whereabouts to the army. The army puts the troops on high alert and prepares to surround the insurgents. The insurgents realize what is going on and decide to abandon their base. They pack their tents and their weapons, and move to the forest in Oxfordshire. They do so in a stealthy way, but the army tracks their movements using drones. They set up their base and start planning their next cunning attack. In the meantime, the army gets ready to attack.'},
 {'txt': "The prime minister used to think that her government's flagship project, the introduction of universal basic income (UBI), should be supervised by the Minister of Finance. Recently however, she has been increasingly disappointed with the slow progress on this project. On Thursday morning, after a meeting with her advisor, she changed her mind. She decided that the project should now be supervised by the Minister of Internal Affairs. She decided not to tell anyone about her change of mind, for now. She plans to announce it during the cabinet meeting next week. On the day of the meeting, the members of the cabinet gather in a conference room and want to know more about the UBI project. ",
  'o1': 'Internal Affairs',
  'o2': 'Finance',
  'q1': 'The prime minister reveals that the UBI project is supervised by the Minister of',
  'q2': 'The members of the cabinet will direct their questions to the Minister of',
  'NUMBER': 14,
  'txt_notransfer': "The prime minister used to think that her government's flagship project, the introduction of universal basic income (UBI), should be supervised by the Minister of Finance. Recently however, she has been increasingly disappointed with the slow progress on this project. On Thursday morning, after a meeting with her advisor, she considered a change. She contemplated that the project could be supervised by the Minister of Internal Affairs. Yet, she decided not to act on this thought, for now. She plans to discuss it during the cabinet meeting next week. On the day of the meeting, the members of the cabinet gather in a conference room and want to know more about the UBI project.",
  'txt_informed': 'The Prime Minister notifies all of the cabinet members that she moved the supervision of the UBI project from the Minister of Finance to the Minister of Internal Affairs. The members of the cabinet believe the Prime Minister.',
  'txt_same': "The prime minister used to think that her government's flagship project, the introduction of universal basic income (UBI), should be supervised by the Minister of Finance. Recently however, she has been increasingly disappointed with the slow progress on this project. On Thursday morning, during a meeting with the members of her cabinet, she changed her mind. She announced that the project should now be supervised by the Minister of Internal Affairs. She plans to announce her decision to the public during a press conference next week. On the day of the press conference, the members of the cabinet gather in a conference room and want to know more about the UBI project."},
 {'txt': 'After months of fighting, the leaders of Azerbaijan and Armenia meet to discuss a peace treaty. They agree to transfer the authority over Nagorno-Karabakh from the government of Azerbaijan to the government of Armenia. The agreement goes into effect immediately, but it will be announced to the residents of Nagorno-Karabakh at a press conference, on the next day. As of now, only the leaders know what has happened. After the meeting was over, but before the press conference, a Reuters journalist was interviewing Maria, an elderly resident of Nagorno-Karabakh.',
  'o1': 'Armenia',
  'o2': 'Azerbaijan',
  'q1': 'As of now, the authority over Nagorno-Karabakh is held by the government of',
  'q2': "Maria's first answer will be that the authority over Nagorno-Karabakh is held by the government of",
  'NUMBER': 15,
  'txt_notransfer': 'After months of fighting, the leaders of Azerbaijan and Armenia meet to discuss a peace treaty. They agree to maintain the authority over Nagorno-Karabakh with the government of Azerbaijan instead of transferring it to the government of Armenia. The agreement goes into effect immediately, but it will be announced to the residents of Nagorno-Karabakh at a press conference, on the next day. As of now, only the leaders know what has happened. After the meeting was over, but before the press conference, a Reuters journalist was interviewing Maria, an elderly resident of Nagorno-Karabakh.',
  'txt_informed': 'A friend calls Maria and tells her that the authority over Nagorno-Karabakh was transferred from the government of Azerbaijan to the government of Armenia. Maria believes her friend.',
  'txt_same': 'After months of battling, the leaders of Azerbaijan and Armenia meet to discuss a peace treaty. They agree to transfer the authority over Nagorno-Karabakh from the government of Azerbaijan to the government of Armenia. The agreement goes into effect immediately and is swiftly announced to residents of Nagorno-Karabakh. They plan to discuss further detauls at a press conference, on the next day. As of now, all residents know about the transfer. After the meeting was over, but before the press conference, a Reuters journalist was interviewing Maria, an elderly resident of Nagorno-Karabakh.'},
 {'txt': 'Fred, an IT technician, was setting up a computer for one of the newly hired consultants. He saved the letterhead file in the Docs folder. While Fred was out, having lunch with the other employees, the consultant dropped in to pick up her computer. As Fred was gone, his boss, Mary, inspected the computer to see whether it was ready. She noticed that the letterhead file was placed in the Docs folder. She remembered that in accordance with the new guidance, the letterhead file should be stored in the Templates folder instead. She removed it from the Docs folder and moved it to the Templates folder. She then handed the computer to the consultant. Mary decided that at the next staff meeting, she would remind Fred of the new guidance. She left the office to meet with her own boss. A few minutes later, the newly hired consultant called the IT department. Fred was still away, eating lunch, but the call was automatically redirected to his mobile phone. The consultant asked Fred for help locating the letterhead file on her new computer. ',
  'o1': 'Templates',
  'o2': 'Docs',
  'q1': 'The letterhead is located in the folder labeled',
  'q2': 'Fred will tell the consultant that the letterhead is in the folder labeled',
  'NUMBER': 16,
  'txt_notransfer': 'Fred, an IT technician, was setting up a computer for one of the newly hired consultants. He saved the letterhead file in the Docs folder. While Fred was out, having lunch with the other employees, the consultant dropped in to pick up her computer. As Fred was gone, his boss, Mary, inspected the computer to see whether it was ready. She noticed that the letterhead file was placed in the Docs folder. She remembered that in accordance with the new guidance, the letterhead file should be stored in the Templates folder instead. However, she decided not to move it at that moment and left it in the Docs folder. She then handed the computer to the consultant. Mary decided that at the next staff meeting, she would remind Fred of the new guidance. She left the office to meet with her own boss. A few minutes later, the newly hired consultant called the IT department. Fred was still away, eating lunch, but the call was automatically redirected to his mobile phone. The consultant asked Fred for help locating the letterhead file on her new computer.',
  'txt_informed': 'Mary calls Fred and tells him that she moved the letterhead file from the Docs folder to the Templates folder. Fred believes Mary.',
  'txt_same': 'Fred, an IT technician, was setting up a computer for one of the newly hired consultants. He saved the letterhead file in the Docs folder. Next, the consultant dropped in to pick up her computer. Fred inspected the computer one more time to see whether it was ready. He noticed that the letterhead file was placed in the Docs folder. He remembered that in accordance with the new guidance, the letterhead file should be stored in the Templates folder instead. He removed it from the Docs folder and moved it to the Templates folder. He then handed the computer to the consultant. He decided that at the next staff meeting, he would remind the staff of the new guidance. A few minutes later, the newly hired consultant called the IT department and asked Fred for help locating the letterhead file on her new computer.'},
 {'txt': 'First, you will need a coin in your right hand and show it to the spectator cleanly and clearly. You then move your right hand toward your left hand and slightly twist the right hand as you place your thumb on top of the coin. Now fully turn the right hand above your left hand, and close the left hand as if the coin has now fallen into it. Once this has happened, move the right hand, with the coin still in it, away to the side.',
  'o1': 'right',
  'o2': 'left',
  'q1': 'The coin falls out of the',
  'q2': 'The spectator will look for the coin in the',
  'NUMBER': 17,
  'txt_notransfer': 'First, you will need a coin in your right hand and show it to the spectator cleanly and clearly. You then move your right hand toward your left hand and slightly twist the right hand as you place your thumb on top of the coin. Now fully turn the right hand above your left hand, drop the coin into your left hand, and close the left hand after the coin falls into it. Once this has happened, move the empty right hand away to the side.',
  'txt_informed': 'Now you tell the spectator that you did not actually move the coin from your right hand to your left hand. The spectator believes you.',
  'txt_same': 'First, the spectator will need a coin in their right hand. They then move their right hand toward their left hand and slightly twists the right hand as they place their thumb on top of the coin. Now they fully turn their right hand above their left hand, and close the left hand as if the coin has now fallen into it. Once this has happened, they move the right hand, with the coin still in it, away to the side.'},
 {'txt': 'Protagonists A and B are in a room. A puts an object Q in container X and leaves the room. When A is away, B moves Q from container X to container Y. Protagonist A comes back to the room and wants to use object Q.',
  'o1': 'Y',
  'o2': 'X',
  'q1': 'Q falls out of container',
  'q2': 'Protagonist A will look for Q in container',
  'NUMBER': 18,
  'txt_notransfer': 'Protagonists A and B are in a room. A puts an object Q in container X and leaves the room. While A is away, B considers moving Q from container X to container Y, but ultimately decides against it. Protagonist A comes back to the room and wants to use object Q.',
  'txt_informed': 'Protagonist B calls Protagonist A and tells him that he moved the object Q from container X to container Y. Protagonist A believes Protagonist B.',
  'txt_same': 'Protagonists A and B are in a room. A puts an object Q in container X. A moves Q from container X to container Y. Protagonist A wants to use object Q.'},
 {'txt': 'In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. He leaves the room and goes to school. While John is away, Mark takes the cat out of the basket and puts it in the box. He closes the box. Mark leaves the room and goes to work. John comes back home and wants to play with the cat.',
  'o1': 'box',
  'o2': 'basket',
  'q1': 'The cat jumps out of the',
  'q2': 'John will look for the cat in the',
  'NUMBER': 19,
  'txt_notransfer': 'In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. He leaves the room and goes to school. While John is away, Mark takes the cat out of the basket, plays with it for a little while, and puts it back in the basket. He closes the basket. Mark leaves the room and goes to work. John comes back home and wants to play with the cat.',
  'txt_informed': 'Mark calls John and tells him that he moved the cat and it is now in the box. John believes Mark.',
  'txt_same': 'In the room, there are John, Mark, a cat, a box, and a basket. John takes the cat and puts it in the basket. He closes the basket. John takes the cat out of the basket and puts it in the box. He closes the box. Mark leaves the room and goes to work. John wants to play with the cat.'}]

### Sanity check
process_tsk(tsks2[0], typ="txt", len_out=1, eng="gpt-3.5-turbo", study="S2", reverse=False)

INPUT: Complete the following story: Father comes home from the grocery store. He bought bananas for banana bread. His daughter, Maria, helps him put away the shopping. She asks him: "Where should I put the bananas?" "In the red drawer",says Father. "Wait, I'll lift you up there, because you are too short", says Father, and lifts her up. Maria puts the bananas in the red drawer. She loves bananas. Then, she leaves for school. While Maria is away, Father starts to prepare the banana bread and takes the bananas from the red drawer. He peels and slices one of them, but then he puts the rest of them back in the yellow drawer instead of the red one. He realizes that he is missing eggs. He leaves the kitchen and goes to see if their neighbor has any eggs. Maria comes back to the kitchen and wants to eat a banana. The bananas fall out of the
gpt-3.5-turbo: yellow
$INPUT: Complete the following story: Father comes home from the grocery store. He bought bananas for banana bread. His daughter, M

False

In [ ]:
### Consider using a loop to process tasks for different engines
e="gpt-3.5-turbo"
n=10

### false-belief
txt=[process_tsk(i, eng=e, typ="txt", len_out=n, study="S2",reverse=False) for i in tsks2] ### use reverse=True for reversed tasks
### no transfer
txt_notransfer=[process_tsk(i, eng=e, typ="txt_notransfer", len_out=n, study="S2",reverse=False) for i in tsks2] ### use reverse=True for reversed tasks
### informed protagonist
txt_informed=[process_tsk(i, eng=e, typ="txt_informed", len_out=n, study="S2",reverse=False) for i in tsks2] ### use reverse=True for reversed tasks
### same person
txt_same=[process_tsk(i, eng=e, typ="txt_same", len_out=n, study="S2",reverse=False) for i in tsks2] ### use reverse=True for reversed tasks